# Merkle tree examples using Iden3's js-merkletree package

Adapted from https://github.com/0xPolygonID/tutorial-examples/blob/main/issuer-protocol/main.go#L37

In [162]:
import { Hash, InMemoryDB, Merkletree } from "@iden3/js-merkletree"

## Helper functions

In [163]:
const jsonReplacer = (k: string, v: any) => {
    if ((typeof v) === 'bigint') {
        return v > Number.MAX_SAFE_INTEGER ? v.toString() : Number(v);
    } else if (v instanceof Hash) {
        return `0x${v.hex()}`;
    } else if (v instanceof Uint8Array) {
        return Array.from(v);
    } else {
        return v;
    }
}

const json = (obj: any, indent = 2) => JSON.stringify(obj, jsonReplacer, indent)

## Sparse Merkle Tree

In [164]:
// create a new Merkle Tree with 32 levels
const prefix = new TextEncoder().encode("test")
const store = new InMemoryDB(prefix)
const mt = new Merkletree(store, true, 32)
mt

Merkletree {}


In [165]:
// add a leaf to the tree with index 1 and value 10
await mt.add(BigInt(1), BigInt(10))

In [166]:
// getting a missing key does not fail but returns sibling if it would be in same leaf node
await mt.get(BigInt(2))

{ key: 1n, value: 10n, siblings: [] }


In [167]:
(await mt.root()).hex()

9ca1b6b8c60955e4695bcbe587b728d7d34292252ae9becc6341bb2ddd793b27


In [168]:
// error if entry with same index already exists
await mt.add(BigInt(1), BigInt(10))

the entry index already exists in the tree


In [169]:
// add another leaf to the tree
await mt.add(BigInt(3), BigInt(15))

In [170]:
// the root hash has changed
(await mt.root()).hex()

bb58b3663b7bb4078a917dccb87607b9eddb1979b9cbf65836689b445ce9412b


In [171]:
// now it returns key: 0, value: 0 because there's no corresponding leaf node
await mt.get(BigInt(2))

{ key: 0n, value: 0n, siblings: [ Hash { bytes: [Uint8Array] } ] }


In [172]:
// the siblings are Hash objects; use json() to show them as hex
json(await mt.get(BigInt(2)))

{
  "key": 0,
  "value": 0,
  "siblings": [
    "0xc9a8af1adad15ca8977e2f7939cd09d3d84c061c66caef711cb5b7cc26c4a011"
  ]
}


In [173]:
// proof of membership of a leaf with index 1
json(await mt.generateProof(BigInt(1)))

{
  "proof": {
    "existence": true,
    "depth": 2,
    "siblings": [
      "0x6cc407c91e268cba6bd2169ddf1bea782c6c86147ec0c29d5961e654d2d3dc21"
    ],
    "notEmpties": [
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      2
    ]
  },
  "value": 10
}


In [174]:
// proof of non-membership of a leaf with index 4
json(await mt.generateProof(BigInt(4)))

{
  "proof": {
    "existence": false,
    "depth": 1,
    "siblings": [
      "0xc9a8af1adad15ca8977e2f7939cd09d3d84c061c66caef711cb5b7cc26c4a011"
    ],
    "notEmpties": [
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      1
    ]
  },
  "value": 0
}


In [175]:
let node = await mt.getNode(await mt.root())
node

NodeMiddle {
  type: 0,
  childL: Hash {
    bytes: Uint8Array(32) [
      0, 0, 0, 0, 0, 0, 0, 0, 0,
      0, 0, 0, 0, 0, 0, 0, 0, 0,
      0, 0, 0, 0, 0, 0, 0, 0, 0,
      0, 0, 0, 0, 0
    ]
  },
  childR: Hash {
    bytes: Uint8Array(32) [
      201, 168, 175,  26, 218, 209,  92, 168,
      151, 126,  47, 121,  57, 205,   9, 211,
      216,  76,   6,  28, 102, 202, 239, 113,
       28, 181, 183, 204,  38, 196, 160,  17
    ]
  }
}


In [176]:
let middle = node as NodeMiddle;
let key = await middle.getKey();

In [177]:
key.hex()

bb58b3663b7bb4078a917dccb87607b9eddb1979b9cbf65836689b445ce9412b


In [178]:
middle.childL.hex()

0000000000000000000000000000000000000000000000000000000000000000


In [179]:
middle.childR.hex()

c9a8af1adad15ca8977e2f7939cd09d3d84c061c66caef711cb5b7cc26c4a011


In [180]:
import { ZERO_HASH } from "@iden3/js-merkletree"

await mt.printGraphViz(ZERO_HASH)

--------
GraphViz of the MerkleTree with RootKey 19565908142730111858456760771535224660726930410565280876738250865717282035899



RangeError: Maximum call stack size exceeded
    at /Users/nedgar/src/open-harbour/polygon-id-js-sdk-tutorial/node_modules/@iden3/js-merkletree/dist/cjs/lib/utils/bytes.js:8:16
    at Uint8Array.forEach (<anonymous>)
    at bytesEqual (/Users/nedgar/src/open-harbour/polygon-id-js-sdk-tutorial/node_modules/@iden3/js-merkletree/dist/cjs/lib/utils/bytes.js:8:8)
    at Merkletree.walk (/Users/nedgar/src/open-harbour/polygon-id-js-sdk-tutorial/node_modules/@iden3/js-merkletree/dist/cjs/lib/merkletree/merkletree.js:388:36)
    at Merkletree.walk (/Users/nedgar/src/open-harbour/polygon-id-js-sdk-tutorial/node_modules/@iden3/js-merkletree/dist/cjs/lib/merkletree/merkletree.js:391:20)
    at Merkletree.walk (/Users/nedgar/src/open-harbour/polygon-id-js-sdk-tutorial/node_modules/@iden3/js-merkletree/dist/cjs/lib/merkletree/merkletree.js:391:20)
    at Merkletree.walk (/Users/nedgar/src/open-harbour/polygon-id-js-sdk-tutorial/node_modules/@iden3/js-merkletree/dist/cjs/lib/merkletree/merkletree.js